In [ ]:
% run 1-datasource.ipynb

In [ ]:
import torch.utils.data as data_utils
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, log_loss
from sklearn.preprocessing import StandardScaler
from tensorboardX import SummaryWriter

In [ ]:
def tensorify(data):
    return torch.from_numpy(data.reshape(-1, 1, 75, 75)).float()

In [ ]:
icebergs.head()

In [ ]:
train, test = train_test_split(icebergs, test_size=0.1, stratify=icebergs.is_iceberg)

In [ ]:
scaler_1 = StandardScaler()
scaler_2 = StandardScaler()

In [ ]:
for _, i in train.iterrows():
    scaler_1.partial_fit(i.band_1.reshape(1, -1))
    scaler_2.partial_fit(i.band_2.reshape(1, -1))

In [ ]:
train_x = tensorify(scaler_1.transform(np.stack(train.band_1)))
test_x = tensorify(scaler_1.transform(np.stack(test.band_1)))
train_y = torch.from_numpy(train.is_iceberg.values.reshape(-1, 1)).long()
test_y = torch.from_numpy(test.is_iceberg.values.reshape(-1, 1)).long()

In [ ]:
cuda = torch.cuda.is_available()
num_epochs = 1
batch_size = 25
learning_rate = 0.0001
cuda

In [ ]:
train_loader = data_utils.DataLoader(data_utils.TensorDataset(train_x, train_y), batch_size=batch_size, shuffle=True)
test_loader = data_utils.DataLoader(data_utils.TensorDataset(test_x, test_y), batch_size=batch_size, shuffle=True)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            #nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            #nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(32 * 18 * 18, 600),
            nn.ReLU(),
            nn.Linear(600, 200),
            nn.ReLU(),
            nn.Linear(200, 2),
            nn.Softmax(1),
        )

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
writer = SummaryWriter()
writer.add_graph(net, net(Variable(train_x.narrow(0, 0, 1))))

if cuda:
    net.cuda()
    criterion.cuda()
    
epoch_train_loss = []
epoch_test_loss = []

In [ ]:
def loop(loader, training=False):
    
    running_loss = 0
    targets = []
    predictions = []
    
    for batch_id, (x, y) in enumerate(loader):
        x = Variable(x)
        y = Variable(y).squeeze()
        
        if cuda:
            x = x.cuda()
            y = y.cuda()
        
        optimizer.zero_grad()
        
        outputs = net(x)
        loss = criterion(outputs, y)

        if training:
            loss.backward()
            optimizer.step()

        running_loss += loss.data.cpu()[0]
        targets.extend(y.data.cpu().numpy())
        predictions.extend(outputs.data.cpu().numpy())
        
    return np.array(targets), np.array(predictions), running_loss

In [ ]:
for e in range(20):

    train_targets, train_preds, train_loss = loop(train_loader, training=True)
    writer.add_scalar('data/train_loss', train_loss, e)
    writer.add_scalar('data/train_accuracy', accuracy_score(train_targets, train_preds.argmax(axis=1)), e)
      
    test_targets, test_preds, test_loss = loop(test_loader, training=False)
    writer.add_scalar('data/test_loss', test_loss, e)
    writer.add_scalar('data/test_accuracy', accuracy_score(test_targets, test_preds.argmax(axis=1)), e)

    train_loss /= train.shape[0]
    test_loss /= test.shape[0]
    
    epoch_train_loss.append(train_loss)
    epoch_test_loss.append(test_loss)
    
    print('Training loss: {:.4f}'.format(train_loss))
    print('Testing  loss: {:.4f}'.format(test_loss))

In [ ]:
plt.plot(epoch_train_loss, label='train')
plt.plot(epoch_test_loss, label='test')
plt.legend();

In [ ]:
train_targets, train_preds, train_loss = loop(train_loader)
accuracy_score(train_targets, train_preds.argmax(axis=1))

In [ ]:
log_loss(train_targets, train_preds)

In [ ]:
test_targets, test_preds, test_loss = loop(test_loader)
accuracy_score(test_targets, test_preds.argmax(axis=1))

In [ ]:
log_loss(test_targets, test_preds)